In [53]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
from pathlib import Path
import json
import time
from typing import Dict, List

openai.api_type = "azure"
openai.api_version = "2022-12-01"

# openai.api_base = "https://turing.openai.azure.com/"
# openai.api_key = os.getenv("OPENAI_API_KEY") or "19923ed0354b4e7e95cec26783f10128"

openai.api_base = "https://baudot.openai.azure.com/"
openai.api_key = os.getenv("OPENAI_API_KEY") or "6ded606d529541778b9a968a4a0b6c6c"

# engine="code-davinci-002-py"

In [47]:
def make_prompt(docstring, fn_def):
    instructions = """
What MicroPython type is returned according to the following docstring, written as a python type hint.
"""
# Take care to correctly type coroutines as Coroutine
    marker = "\n\n\n-=-+-=-\n"
    prompt = f"""{instructions}\n\n{fn_def}\n    '''\n{docstring}\n    '''{marker}"""
    return prompt


def get_type(docstring,fn_def = "", engine="text-davinci-002", verbose=False):

    prompt = make_prompt(docstring, fn_def)

    if verbose:
        print(prompt)
    try:
        # Create a completion for the provided prompt and parameters
        # To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
        completion = openai.Completion.create(
                        prompt=prompt,
                        temperature=0,
                        max_tokens=30,
                        engine=engine,
                        stop=["# ", "import ", "return", "def"] 
                        )

        return_type = completion.choices[0].text.strip(" \n")
        
        # Here indicating if the response is filtered
        if completion.choices[0].finish_reason == "content_filter":
            print("The generated content is filtered.")

        return return_type
            
    except openai.error.APIError as e:
        # Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        return None

    except openai.error.AuthenticationError as e:
        # Handle Authentication error here, e.g. invalid API key
        print(f"OpenAI API returned an Authentication Error: {e}")
        return None

    except openai.error.APIConnectionError as e:
        # Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
        return None

    except openai.error.InvalidRequestError as e:
        # Handle connection error here
        print(f"Invalid Request Error: {e}")
        return None

    except openai.error.RateLimitError as e:
        # Handle rate limit error
        print(f"OpenAI API request exceeded rate limit: {e}")
        return None

    except openai.error.ServiceUnavailableError as e:
        # Handle Service Unavailable error
        print(f"Service Unavailable: {e}")
        return None

    except openai.error.Timeout as e:
        # Handle request timeout
        print(f"Request timed out: {e}")
        return None





In [55]:
docstring = """
Close the database. It's mandatory to close the database at the end of,
processing, as some unwritten data may be still in the cache. Note that
this does not close underlying stream with which the database was opened,
it should be closed separately (which is also mandatory to make sure that
data flushed from buffer to the underlying storage).
"""

print (get_type(docstring, engine="gpt-35", verbose=True))


What MicroPython type is returned according to the following docstring, written as a python type hint.



    '''

Close the database. It's mandatory to close the database at the end of,
processing, as some unwritten data may be still in the cache. Note that
this does not close underlying stream with which the database was opened,
it should be closed separately (which is also mandatory to make sure that
data flushed from buffer to the underlying storage).

    '''


-=-+-=-

None

-=-+-=-

<|im_end|>


In [49]:


engine="text-davinci-003"

filename = Path("./tests/rst/data/return_testcases.json")
doc = []
with open(filename, encoding="utf8") as fp:
    doc = json.load(fp)

dataset:List[Dict] = []

for tc in doc:
    # if not "Coroutine" in tc["type"] :
    #     continue
    try:
        docstring = '\n'.join(tc["docstring"])
        fn_def = f"def {tc['signature']}:"
        prompt = make_prompt(docstring, fn_def)
        ai_type = get_type(docstring, fn_def=fn_def, engine=engine, verbose=False)
        dataset.append({"prompt": prompt, "completion": tc["type"], "completion2": ai_type})
        time.sleep(0.5)
    except KeyError:
        print("INVALID TEST DATA ERROR", tc)

# write out the dataset
with open("return_testcases.json", "w") as fp:
    json.dump(dataset, fp, indent=4)
    


## Training data


In [51]:
"""
Build a training set bases on the test cases and compare with the expected results and AI results
to allow manual editing of the training set.
"""



# from pathlib import Path
# import json
# import time

# filename = Path("./tests/rst/data/return_testcases.json")
# doc = []
# with open(filename, encoding="utf8") as fp:
#     doc = json.load(fp)
# cases = []

# for tc in doc:
#     # if not "Coroutine" in tc["type"] :
#     #     continue
#     try:
#         docstring = '\n'.join(tc["docstring"])
#         fn_def = f"def {tc['signature']}:"
#         ai_type = get_type(docstring, fn_def=fn_def, engine=engine, verbose=False)
#         # ai_type = get_type(docstring, fn_def=fn_def, engine="text-curi-003", verbose=True)
#         print(f"{tc['signature']} -> {tc['type']}:\n{tc['signature']} -> {ai_type}:")
#         print( "."*80)
        
#         time.sleep(0.5)

#     except KeyError:
#         print("INVALID TEST DATA ERROR", tc)



What MicroPython type is returned according to the following docstring, written as a python type hint.


def open(stream, *, flags=0, pagesize=0, cachesize=0, minkeypage=0):
    '''
   Open a database from a random-access `stream` (like an open file). All
   other parameters are optional and keyword-only, and allow to tweak advanced
   parameters of the database operation (most users will not need them):

   * *flags* - Currently unused.
   * *pagesize* - Page size used for the nodes in BTree. Acceptable range
     is 512-65536. If 0, a port-specific default will be used, optimized for
     port's memory usage and/or performance.
   * *cachesize* - Suggested memory cache size in bytes. For a
     board with enough memory using larger values may improve performance.
     Cache policy is as follows: entire cache is not allocated at once;
     instead, accessing a new page in database will allocate a memory buffer
     for it, until value specified by *cachesize* is reached. Then, these


KeyboardInterrupt: 